In [115]:
'''
This notebook is oriented to use a given dataset customized to contains 6 features and training a classification tree
This features are: cqt, rmse, energy, mfccs, chromagram and spectral contrast
This notebook is currently in development phase, so contains a considerably quantity of test code

'''

'\nThis notebook is oriented to use a given dataset customized to contains 6 features and training a classification tree\nThis features are: cqt, rmse, energy, mfccs, chromagram and spectral contrast\nThis notebook is currently in development phase, so contains a considerably quantity of test code\n\n'

In [55]:
import tensorflow as tf
import pandas as pd
from pathlib import Path
import librosa, librosa.display
import matplotlib.pyplot as plt
import numpy as np
import gc

# Reading Audio Data

# features = pd.read_csv('Datasets/2000Spoti.csv', encoding='latin-1', thousands=',')

# features = pd.read_csv('Dataframes/var_mean_dict.csv', encoding='latin-1', thousands=',') # Media genera 0.62-0.67 con todas las caracteristicas **best result2**
features = pd.read_csv('Dataframes/sliceDataframe/var_mean_dict_10000_Samples.csv', encoding='latin-1', thousands=',') # Media genera 0.62-0.67 con todas las caracteristicas **best result2**

print(features.head(5))

print('done')

   Unnamed: 0  Genre  Energy_Mean  RMSE_Mean  ZCR_Mean  CQT_Mean  \
0           0  Blues    42.006336   0.130295      5498  0.024378   
1           1  Blues    28.860627   0.112494      5840  0.006032   
2           2  Blues    44.372582   0.131844      4781  0.012576   
3           3  Blues    39.969162   0.132396      4643  0.014717   
4           4  Blues    44.813606   0.143477      4699  0.035051   

   Spectral_Contrast_Mean  Chromagram_Mean  Tempo_Mean  STFT_Mean  ...  \
0               20.876659         0.278372  129.199219   0.000074  ...   
1               20.660786         0.275674  123.046875   0.000080  ...   
2               20.529431         0.273605  123.046875   0.000058  ...   
3               20.406874         0.280773  123.046875   0.000039  ...   
4               20.370873         0.273082  123.046875   0.000046  ...   

   MFCCS3  MFCCS4  MFCCS5  MFCCS6  MFCCS7  MFCCS8  MFCCS9  MFCCS10  MFCCS11  \
0     337     114     187      64      44      51      34       43 

In [56]:

#extracting labels and features

labelColumn = 'Genre'

labels = np.array(features[labelColumn])
print(np.unique(labels))

print("The labels of out dataset are:", np.unique(labels))
print("And their shape is:", labels.shape)


# Only mfcc
            
# featureColumsToDrop = ['Unnamed: 0', 'STFT_Mean.1', 'STFT_Var.1', 'Genre', 'Energy_Mean', 'RMSE_Mean', 'ZCR_Mean', 'CQT_Mean', 
#                            'Spectral_Contrast_Mean', 'Chromagram_Mean', 'Tempo_Mean', 'STFT_Mean', 
#                           'Autocorrelation_Mean', 'STFT_Mean', 'Spectral_Centroid_Mean', 'Rolloff_Mean',
#                           'Energy_Var', 'RMSE_Var', 'ZCR_Var', 'CQT_Var', 'Spectral_Contrast_Var', 'Chromagram_Var', 'Tempo_Var', 'STFT_Var',
#                           'Autocorrelation_Var', 'STFT_Var', 'Spectral_Centroid_Var', 'Rolloff_Var']

# ALL

# featureColumsToDrop = ['Unnamed: 0', 'STFT_Mean.1', 'STFT_Var.1', 'Genre']

# Feture importance ordered

featureColumsToDrop = ['Unnamed: 0', 'STFT_Mean.1', 'STFT_Var.1', 'Genre',
                        'Energy_Mean', 'RMSE_Mean', 'ZCR_Mean', 'CQT_Mean', 'Tempo_Mean',
                        'ZCR_Var', 'Spectral_Contrast_Var','Tempo_Var', 'STFT_Var',
                        'MFCCS0','MFCCS1','MFCCS2','MFCCS3','MFCCS4','MFCCS5','MFCCS6','MFCCS7',
                        'MFCCS8','MFCCS9','MFCCS10', 'CQT_Mean','MFCCS11','MFCCS12']

# NO MFCCS mean and var

# featureColumsToDrop = ['Unnamed: 0', 'STFT_Mean.1', 'STFT_Var.1', 'Genre',
#                        'Energy_Mean', 'RMSE_Mean', 'ZCR_Mean', 'CQT_Mean', 'Spectral_Contrast_Mean', 'Chromagram_Mean', 'Tempo_Mean', 'STFT_Mean', 
#                       'Autocorrelation_Mean', 'STFT_Mean', 'Spectral_Centroid_Mean', 'Rolloff_Mean',
#                        'MFCCS0','MFCCS1','MFCCS2','MFCCS3','MFCCS4','MFCCS5','MFCCS6','MFCCS7','MFCCS8','MFCCS9','MFCCS10', 'CQT_Mean','MFCCS11','MFCCS12', 'CQT_Var', 'STFT_Var.1', 'Genre']  


# First prove that was made

# featureColumsToDrop = ['Unnamed: 0', 'Autocorrelation_Mean', 'STFT_Mean', 'Rolloff_Mean', 'Energy_Var', 'RMSE_Var', 'ZCR_Var', 'CQT_Var', 'Spectral_Contrast_Var', 'Chromagram_Var', 'Tempo_Var', 'STFT_Var',
#                       'Autocorrelation_Var', 'STFT_Var', 'Spectral_Centroid_Var', 'Rolloff_Var', 'CQT_Mean', 'STFT_Mean.1', 'STFT_Var.1', 'Genre']

features = features.drop(featureColumsToDrop, 1)

print("The features of out dataset are:")

features.head(5)

['Blues' 'Classical' 'Country' 'Disco' 'Hiphop' 'Jazz' 'Metal' 'Pop'
 'Reggae' 'Rock']
The labels of out dataset are: ['Blues' 'Classical' 'Country' 'Disco' 'Hiphop' 'Jazz' 'Metal' 'Pop'
 'Reggae' 'Rock']
And their shape is: (10000,)
The features of out dataset are:


C:\Users\alvar\anaconda3\envs\tfg\lib\site-packages\ipykernel_launcher.py:45: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


,Spectral_Contrast_Mean,Chromagram_Mean,STFT_Mean,Autocorrelation_Mean,Spectral_Centroid_Mean,Rolloff_Mean,Energy_Var,RMSE_Var,CQT_Var,Chromagram_Var,Autocorrelation_Var,Spectral_Centroid_Var,Rolloff_Var
0,20.876659,0.278372,0.000074,58.393989,1773.285877,3689.714355,1420.957631,0.003541,0.319991,0.005842,113157.328633,168244.728448,1.084507e+06
1,20.660786,0.275674,0.000080,64.492910,1816.195860,3838.541917,405.017719,0.001466,0.285368,0.007337,60033.748726,90703.325185,6.702624e+05
2,20.529431,0.273605,0.000058,51.091079,1788.642783,3955.152494,1708.942772,0.004602,0.392569,0.008474,243928.138751,111322.537051,7.869501e+05
3,20.406874,0.280773,0.000039,60.225847,1654.902168,3530.617112,703.118149,0.002423,0.318879,0.004500,111004.949290,112316.264385,9.115652e+05
4,20.370873,0.273082,0.000046,73.685011,1630.737017,3439.432279,652.843329,0.001724,0.388950,0.008759,154556.712147,79648.228297,6.096879e+05


In [57]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from matplotlib.pyplot import figure
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers

def train_clasify():
    """
    Method that train a RF Classifier and return a metrics report with
        - Recall by label
        - Precision by label
        - F1-score by label
        - Avg weighted recall
        - Avg weighted precision
        - Avg weighted f1-score
        - Avg macro recall
        - Avg macro recall
        - Avg macro recall
        - Avg total accuracy
    """

    # Split the data into training and testing sets
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25)

    print('Training Features Shape:', train_features.shape)
    print('Training Labels Shape:', train_labels.shape)
    print('Testing Features Shape:', test_features.shape)
    print('Testing Labels Shape:', test_labels.shape)

    scaler = StandardScaler()
    scaler.fit(train_features)
    train_features_scaled = scaler.transform(train_features)
    test_features_scaled =  scaler.transform(test_features)
    
    print('Train Features Scaled Shape:', train_features_scaled.shape)
    print('Testing Features Scaled Shape:', test_features_scaled.shape)
    
    # Instantiate model with 1000 decision trees
    rf = RandomForestClassifier(n_estimators = 1000)
    
    scores = cross_val_score(rf, train_features_scaled, train_labels, cv=5)
    
    # Train the model on training data
    rf.fit(train_features_scaled, train_labels)
    
    print("-------------------------------------------")
    print("Metrics info section")
    print("-------------------------------------------")
    
    print("Scores by fold")
    print(scores)
    
    importancia_predictores = pd.DataFrame({
        'feature':
        features.columns,
        'importance':
        rf.feature_importances_
    })
    print("Feature importance in the model")
    print("-------------------------------------------")
    print(importancia_predictores.sort_values('importance', ascending=False))

    predictions = rf.predict(test_features_scaled)


    report = classification_report(test_labels, predictions, np.unique(labels))
    return report

report = train_clasify()
print(report)

Training Features Shape: (7500, 13)
Training Labels Shape: (7500,)
Testing Features Shape: (2500, 13)
Testing Labels Shape: (2500,)
Train Features Scaled Shape: (7500, 13)
Testing Features Scaled Shape: (2500, 13)
-------------------------------------------
Metrics info section
-------------------------------------------
Scores by fold
[0.806      0.818      0.80266667 0.79866667 0.81333333]
Feature importance in the model
-------------------------------------------
                   feature  importance
8                  CQT_Var    0.090742
0   Spectral_Contrast_Mean    0.087168
5             Rolloff_Mean    0.087079
6               Energy_Var    0.083670
2                STFT_Mean    0.082942
3     Autocorrelation_Mean    0.081573
12             Rolloff_Var    0.078053
10     Autocorrelation_Var    0.077336
4   Spectral_Centroid_Mean    0.077294
7                 RMSE_Var    0.071758
11   Spectral_Centroid_Var    0.069303
9           Chromagram_Var    0.056660
1          Chromagram_

C:\Users\alvar\anaconda3\envs\tfg\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass labels=['Blues' 'Classical' 'Country' 'Disco' 'Hiphop' 'Jazz' 'Metal' 'Pop'
 'Reggae' 'Rock'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [4]:
import junix;
junix.export_images(filepath= "ClassificationTree.ipynb")


In [5]:
plot_confusion_matrix(rf, test_features, test_labels)
fig = plt.gcf()
fig.set_size_inches(28.5, 28.5)
plt.show()

import junix;
junix.export_images(filepath= "ClassificationTree.ipynb")

NameError: name 'rf' is not defined

In [58]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from matplotlib.pyplot import figure
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold


def train_ann_model(inputs, targets):
    # Instantiate model with 1000 decision trees
    model = keras.models.Sequential()
#     model.add(keras.layers.LSTM(24, return_sequences=True, implementation=2))
    model.add(keras.layers.Dense(512, activation='relu', input_shape=((inputs.shape[1],))))
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dense(128, activation='relu'))
#     model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='softmax'))
    
    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    # train
    model_history = model.fit(inputs, targets,
                                    epochs=512,
                                    batch_size=64,
#                                     validation_split=0.2,
#                                     validation_data=(test_features, test_labels),
                                    verbose=False)
    
    model.summary()
    
#     report = classification_report(test_labels, predictions, np.unique(labels))
    
    return model, model_history



def train_clasify():
    """
    Method that train a RF Classifier and return a metrics report with
        - Recall by label
        - Precision by label
        - F1-score by label
        - Avg weighted recall
        - Avg weighted precision
        - Avg weighted f1-score
        - Avg macro recall
        - Avg macro recall
        - Avg macro recall
        - Avg total accuracy
    """
    # Encode 
    
    label_encoder = LabelEncoder()
    label_encoder.fit(labels)
    labels_encoded = label_encoder.transform(labels)

    # Split the data into training and testing sets
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels_encoded, test_size = 0.25)

    print('Training Features Shape:', train_features.shape)
    print('Training Labels Shape:', train_labels.shape)
    print('Testing Features Shape:', test_features.shape)
    print('Testing Labels Shape:', test_labels.shape)
    
    scaler = StandardScaler()
    scaler.fit(train_features)
    train_features_scaled = scaler.transform(train_features)
    test_features_scaled =  scaler.transform(test_features)
    
    # Merge inputs and targets
    inputs = np.concatenate((train_features_scaled, test_features_scaled), axis=0)
    targets = np.concatenate((train_labels, test_labels), axis=0)
    
    # Define the K-fold Cross Validator
    kfold = KFold(n_splits=10, shuffle=True)

    # K-fold Cross Validation model evaluation
    fold_no = 1
    
    for train, test in kfold.split(inputs, targets):
        print(train.shape)
        model, history = train_ann_model(inputs[train], targets[train])
        
        scores = model.evaluate(inputs[test], targets[test], verbose=0)
        print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
#         acc_per_fold.append(scores[1] * 100)
#         loss_per_fold.append(scores[0])
        
        # Increase fold number
        fold_no = fold_no + 1
    

train_clasify()
    
# ann_model = train_clasify()

# ann_model.history

# plt.plot(ann_model.history['accuracy'])

# matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))


Training Features Shape: (7500, 13)
Training Labels Shape: (7500,)
Testing Features Shape: (2500, 13)
Testing Labels Shape: (2500,)
(9000,)
Model: "sequential_81"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_317 (Dense)            (None, 512)               7168      
_________________________________________________________________
dense_318 (Dense)            (None, 256)               131328    
_________________________________________________________________
dense_319 (Dense)            (None, 128)               32896     
_________________________________________________________________
dense_320 (Dense)            (None, 64)                8256      
Total params: 179,648
Trainable params: 179,648
Non-trainable params: 0
_________________________________________________________________
Score for fold 1: loss of 1.0702067613601685; accuracy of 85.6000006198883%
(9000,)
Model: "sequential_82

Model: "sequential_90"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_353 (Dense)            (None, 512)               7168      
_________________________________________________________________
dense_354 (Dense)            (None, 256)               131328    
_________________________________________________________________
dense_355 (Dense)            (None, 128)               32896     
_________________________________________________________________
dense_356 (Dense)            (None, 64)                8256      
Total params: 179,648
Trainable params: 179,648
Non-trainable params: 0
_________________________________________________________________
Score for fold 10: loss of 1.1807438135147095; accuracy of 83.20000171661377%


In [ ]:
"""
This code was develop in order to implement corss validation obtaining more metrics that in sklearn version
"""

def get_metrics(report, tag, start_point):
    """
    Method that extract metrics from the string table returned by the classification_report method from sklearn package
    """
    avg_precision = report[report.index(tag)+start_point:report.index(tag)+start_point+4]
    avg_recall = report[report.index(tag)+start_point+10:report.index(tag)+start_point+10+4]
    avg_f1_score = report[report.index(tag)+start_point+20:report.index(tag)+start_point+20+4]
    return avg_precision, avg_recall, avg_f1_score

def get_metrics_avg(number_iterations):
    """
    Method that make number_iterations training a RF classifier and return 2 dictionaires with metics
    
        - One with metrics by genre label
        - The other with total metrics of all the genre labels
    """
    
    # Initializing the by genre and total metrics arrays necessaries to catch the data
    
    blues_precision_array = np.arange(number_iterations).astype(np.float32)
    blues_recall_array = np.arange(number_iterations).astype(np.float32)
    blues_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    classical_precision_array = np.arange(number_iterations).astype(np.float32)
    classical_recall_array = np.arange(number_iterations).astype(np.float32)
    classical_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    country_precision_array = np.arange(number_iterations).astype(np.float32)
    country_recall_array = np.arange(number_iterations).astype(np.float32)
    country_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    disco_precision_array = np.arange(number_iterations).astype(np.float32)
    disco_recall_array = np.arange(number_iterations).astype(np.float32)
    disco_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    hiphop_precision_array = np.arange(number_iterations).astype(np.float32)
    hiphop_recall_array = np.arange(number_iterations).astype(np.float32)
    hiphop_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    jazz_precision_array = np.arange(number_iterations).astype(np.float32)
    jazz_recall_array = np.arange(number_iterations).astype(np.float32)
    jazz_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    metal_precision_array = np.arange(number_iterations).astype(np.float32)
    metal_recall_array = np.arange(number_iterations).astype(np.float32)
    metal_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    pop_precision_array = np.arange(number_iterations).astype(np.float32)
    pop_recall_array = np.arange(number_iterations).astype(np.float32)
    pop_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    reggae_precision_array = np.arange(number_iterations).astype(np.float32)
    reggae_recall_array = np.arange(number_iterations).astype(np.float32)
    reggae_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    rock_precision_array = np.arange(number_iterations).astype(np.float32)
    rock_recall_array = np.arange(number_iterations).astype(np.float32)
    rock_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    macro_avg_precision_array = np.arange(number_iterations).astype(np.float32)
    macro_avg_recall_array = np.arange(number_iterations).astype(np.float32)
    macro_avg_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    weighted_avg_precision_array = np.arange(number_iterations).astype(np.float32)
    weighted_avg_recall_array = np.arange(number_iterations).astype(np.float32)
    weighted_avg_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    accuracy_avg_array = np.arange(number_iterations).astype(np.float32)
    
    labels = ['Blues','Classical','Country','Disco','Hiphop','Jazz','Metal','Pop','Reggae','Rock']
    
    # Loop with the training RF classiffier itrerations. We fill the arrays metrics in each iteration     
    
    for i in range(0, number_iterations):
        report = train_clasify()
        
        # Metrics by Genre
        (precision, recall, f1_score) = get_metrics(report, "Blues", 12)
        blues_precision_array[i] = precision
        blues_recall_array[i] = recall
        blues_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Classical", 16)
        classical_precision_array[i] = precision
        classical_recall_array[i] = recall
        classical_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Country", 14)
        country_precision_array[i] = precision
        country_recall_array[i] = recall
        country_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Disco", 12)
        disco_precision_array[i] = precision
        disco_recall_array[i] = recall
        disco_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Hiphop", 13)
        hiphop_precision_array[i] = precision
        hiphop_recall_array[i] = recall
        hiphop_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Jazz", 11)
        jazz_precision_array[i] = precision
        jazz_recall_array[i] = recall
        jazz_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Metal", 12)
        metal_precision_array[i] = precision
        metal_recall_array[i] = recall
        metal_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Pop", 10)
        pop_precision_array[i] = precision
        pop_recall_array[i] = recall
        pop_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Reggae", 13)
        reggae_precision_array[i] = precision
        reggae_recall_array[i] = recall
        reggae_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Rock", 11)
        rock_precision_array[i] = precision
        rock_recall_array[i] = recall
        rock_f1_score_array[i] = f1_score
        
        # General metrics         
        
        (precision, recall, f1_score) = get_metrics(report, "accuracy", 15)
        accuracy_avg_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "weighted", 19)
        weighted_avg_precision_array[i] = precision
        weighted_avg_recall_array[i] = recall
        weighted_avg_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "macro avg", 16)
        macro_avg_precision_array[i] = precision
        macro_avg_recall_array[i] = recall
        macro_avg_f1_score_array[i] = f1_score
        
    # Now we fill the arrays that we are going to use to create the final dictionaries     
    
    precision_array = []
    recall_array = []
    f1_score_array = []
    
    precision_array.append(blues_precision_array.mean())
    recall_array.append(blues_recall_array.mean())
    f1_score_array.append(blues_f1_score_array.mean())
    
    precision_array.append(classical_precision_array.mean())
    recall_array.append(classical_recall_array.mean())
    f1_score_array.append(classical_f1_score_array.mean())
    
    precision_array.append(country_precision_array.mean())
    recall_array.append(country_recall_array.mean())
    f1_score_array.append(country_f1_score_array.mean())
    
    precision_array.append(disco_precision_array.mean())
    recall_array.append(disco_recall_array.mean())
    f1_score_array.append(disco_f1_score_array.mean())
    
    precision_array.append(hiphop_precision_array.mean())
    recall_array.append(hiphop_recall_array.mean())
    f1_score_array.append(hiphop_f1_score_array.mean())
    
    precision_array.append(jazz_precision_array.mean())
    recall_array.append(jazz_recall_array.mean())
    f1_score_array.append(jazz_f1_score_array.mean())
    
    precision_array.append(metal_precision_array.mean())
    recall_array.append(metal_recall_array.mean())
    f1_score_array.append(metal_f1_score_array.mean())
    
    precision_array.append(pop_precision_array.mean())
    recall_array.append(pop_recall_array.mean())
    f1_score_array.append(pop_f1_score_array.mean())
    
    precision_array.append(reggae_precision_array.mean())
    recall_array.append(reggae_recall_array.mean())
    f1_score_array.append(reggae_f1_score_array.mean())
    
    precision_array.append(rock_precision_array.mean())
    recall_array.append(rock_recall_array.mean())
    f1_score_array.append(rock_f1_score_array.mean())
    
    # Dictionaries creation     
    
    by_label_metrics_dict = {
        
        'Genre': labels,
        'avg_precision': precision_array,
        'avg_recall': recall_array,
        'avg_f1_score': f1_score_array,
        
    }
    
    total_metrics_dict = {
        
        'avg_accuracy': accuracy_avg_array.mean(),
        
        'weighted_avg_precision': weighted_avg_precision_array.mean(),
        'weighted_avg_recall': weighted_avg_recall_array.mean(),
        'weighted_avg_f1_score': weighted_avg_f1_score_array.mean(),
        
        'macro_avg_precision': macro_avg_precision_array.mean(),
        'macro_avg_recall': macro_avg_recall_array.mean(),
        'macro_avg_f1_score': macro_avg_f1_score_array.mean(),
        
    }

    return by_label_metrics_dict, total_metrics_dict


def export_genre_result_dict_to_csv(by_label_metrics_dict, total_metrics_dict):
    """
    Method that export two dictionaries to csv files using pandas dataframes
    
        - One with metrics by genre label
        - The other with total metrics of all the genre labels
    """
    by_label_metrics_dataframe_columns_tags = ['Genre', 'avg_precision', 'avg_recall', 'avg_f1_score']
    total_metrics_dataframe_columns_tags = ['avg_accuracy', 
                                            'weighted_avg_precision', 'weighted_avg_recall', 'weighted_avg_f1_score',
                                            'macro_avg_precision', 'macro_avg_recall', 'macro_avg_f1_score'
                                           ]

    df_by_label = pd.DataFrame(by_label_metrics_dict, columns = by_label_metrics_dataframe_columns_tags)
    df_total = pd.DataFrame(total_metrics_dict, columns = total_metrics_dataframe_columns_tags, index=[0])

    df_by_label.to_csv('RF_Standarized_byLabel.csv')
    df_total.to_csv('RF_Standarized_total.csv')

